In [1]:
import netCDF4
import xarray as xr
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split

In [ ]:
#NOTE: File paths should be the same if you are working in the repo.
#Also, I would push these csv's but they're bigger than the max file size lmao
wind = xr.open_dataset('../Data/new_raw_data/agg_terraclimate_windspeed_1979_2020.nc')
drought = xr.open_dataset('../Data/new_raw_data/Drought_PDSI_1979_2020_GLOBE.nc')
wind.ws.to_dataframe().to_csv('../Data/new_raw_data/windspeed.csv')
drought.PDSI.to_dataframe().to_csv('../Data/new_raw_data/drought.csv')

In [ ]:
wind_df = pd.read_csv('../data/new_raw_data/windspeed.csv')
wind_df = np.round(wind_df, decimals=1)
wind_df.head(5)

In [ ]:
drought_df = pd.read_csv('../data/new_raw_data/drought.csv')
drought_df = np.round(drought_df, decimals=1)
drought_df.head()

In [ ]:
wildfire_df = pd.read_csv('../Data/Clean Data/Wildfires/cleaned_wildfires.csv')
wildfire_df = np.round(wildfire_df, decimals=1)
wildfire_df.head()

In [ ]:
wildfire_df['DISCOVERY_DATE'].apply(str)
date_list = wildfire_df['DISCOVERY_DATE'].tolist()
fire_list = []
for i in range(len(date_list)):
    fire_list.append(1)
    if not date_list[i].endswith('01'):
        date_list[i] = date_list[i][:-2] + "01"
wildfire_df['time'] = date_list
wildfire_df['fire'] = fire_list
wildfire_df = wildfire_df.drop(['DISCOVERY_DATE'], axis=1)
wildfire_df = wildfire_df.rename({'LATITUDE': 'lat', 'LONGITUDE': 'lon'}, axis=1)
wildfire_df = wildfire_df[['time', 'lat', 'lon', 'fire']]
wildfire_df.head()

In [ ]:
final_df = pd.merge(wind_df, drought_df, how='outer')
final_df = final_df.dropna()
final_df = pd.merge(final_df, wildfire_df, how='outer')
final_df.head()

In [ ]:
final_df.shape

In [ ]:
final_df = final_df[final_df['time'].between('2000-01-01', '2015-12-01')]
# print(final_df.dtypes)
final_df.tail()

In [ ]:
final_df = final_df[final_df['ws'].notna()]
final_df = final_df[final_df['PDSI'].notna()]
final_df.head()

In [ ]:
final_df.shape

In [ ]:
final_df.to_csv("../Data/Clean Data/final_data.csv", encoding='utf-8', index=False)

In [ ]:
# Split up the independent and dependent variables, where the independent variables are the lattitude, longitude, wind speed, and PDSI. The dependent is whether there was a fire or not
x = final_df[['lat','lon','ws', 'PDSI']]
y = final_df['fire']

# Pick what fraction of the data you want to be the testing data
fraction_split = 0.2

In [ ]:
#split the data into training and testing data using sklearn function train_test_split
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=fraction_split)
print(X_train)

In [ ]:
# Output the training and testing data to two csvs in Clean Data folder
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)

test_x = pd.DataFrame(test_x)
test_y = pd.DataFrame(test_y)


training_data = train_x.assign(fire = train_y)
testing_data = test_x.assign(fire = test_y)

print(training_data)
training_data.to_csv("../Data/Clean Data/training_data.csv", encoding='utf-8', index=False)
testing_data.to_csv("../Data/Clean Data/testing_data.csv", encoding='utf-8', index=False)